In [1]:
import glob
import re
import collections
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer

In [2]:
#Read File
files_txt = glob.glob("/opt/datasets/mcda-pi1-20191/papers-txt/*.txt")

wordnet_lemmatizer = WordNetLemmatizer()
sw = stopwords.words("english")

bag_of_words = {}
for file in files_txt:
    input_file = open(file,"r",encoding='utf-8')
    texto = input_file.read()
    texto = re.sub('[^A-Za-z0-9]+',' ',texto) # Caracteres especiales
    tokens = texto.split()
    stemmer = PorterStemmer()
    stemmer2 = LancasterStemmer()
    tokens = [wordnet_lemmatizer.lemmatize(stemmer.stem(w.lower()), pos="v") for w in tokens if (len(w)>1) and w.isalpha() and w not in sw] # Longitud mayor a 1
    counter=collections.Counter(tokens)
    bag_of_words[file] = dict(counter)


In [ ]:
bag_of_words[files_txt[0]]
#sort
import operator
sorted_x = sorted(bag_of_words[files_txt[0]].items(), key=lambda kv: kv[1])
sorted_x

In [ ]:
keys, values = [], []
for key, value in bag_of_words.items():
    keys.append(key)
    values.append(value)
pd.DataFrame(data=values, index=keys)

In [11]:
helper = {}
i = 0
for key, value in bag_of_words.items():
    for word in value.keys():
        if word in helper:
            helper[word] += 1
        else:
            helper[word] = 1
            

#freq 0
#Term frequency 1
#tf 2
#Inverse Document Frequency 3
num_words = sum(bag_of_words[files_txt[0]].values())

for documento, words in bag_of_words.items():
    num_words = sum(bag_of_words[documento].values())
    for key, value in words.items():
        dic = {}
        #print(value)
        dic['freq'] = value
        dic['freqR'] = value / num_words
        dic['tf'] = 1 + np.log(value)
        dic['idf'] = np.log(len(bag_of_words) / helper[key])
        bag_of_words[documento][key] = dic

In [ ]:
#query
def transformar_informacion (texto):
    texto = re.sub('[^A-Za-z0-9]+',' ',texto) # Caracteres especiales
    tokens = texto.split()
    stemmer = PorterStemmer()
    stemmer2 = LancasterStemmer()
    tokens = [wordnet_lemmatizer.lemmatize(stemmer.stem(w.lower()), pos="v") for w in tokens if (len(w)>1) and w.isalpha() and w not in sw] # Longitud mayor a 1
    return tokens

texto = "User friendly user"
query = transformar_informacion(texto)

# Metricas

In [13]:
#Contar documentos
len(bag_of_words)

980

In [27]:
#Contar numero de palabras por documento despues de limpieza
x_doc = [] # documentos
y_doc = [] # num palabras unicas por doc
total_palabras_doc = [] # total palabras
for documento, words in bag_of_words.items():
    x_doc.append(documento)
    y_doc.append(len(words))
    numero_palabras = 0
    for word, values in words.items():
        numero_palabras += values['freq']
    total_palabras_doc.append(numero_palabras)

#Imprimir Solo un documento
print("El documento '{}' tiene {} palabras unicas despues del proceso de limpieza".format(x_doc[0][-15:], y_doc[0]))
print("El total de palabras contenido en el texto es de {}".format(total_palabras_doc[0]))

El documento '/1511.06030.txt' tiene 923 palabras unicas despues del proceso de limpieza
El total de palabras contenido en el texto es de 3747


In [52]:
palabras_raras = collections.OrderedDict()
palabras_tres_caracteres = collections.OrderedDict()
i = 0
for documento, words in bag_of_words.items():
    for word, values in words.items():
        if len(word) > 10 and values['freq'] < 2:
            palabras_raras[i] = word
            i+=1
        if bool(re.search(r'((\w)\2{2,})', word)):
            palabras_tres_caracteres[word] = values['freq']
palabras_raras

OrderedDict([(0, 'characterist'),
             (1, 'activandrew'),
             (2, 'frequentist'),
             (3, 'demonstratt'),
             (4, 'arbitrarili'),
             (5, 'distribulikelihood'),
             (6, 'dominattribut'),
             (7, 'loglikelihood'),
             (8, 'likelihoodbas'),
             (9, 'technologiesvolum'),
             (10, 'salakhutdinov'),
             (11, 'sarrafzadeh'),
             (12, 'distinguish'),
             (13, 'multidisciplinari'),
             (14, 'neighborhood'),
             (15, 'surprisingli'),
             (16, 'necessarili'),
             (17, 'learningbas'),
             (18, 'frequencytun'),
             (19, 'complementari'),
             (20, 'multiresolut'),
             (21, 'heynderickx'),
             (22, 'scandinavian'),
             (23, 'reconstruct'),
             (24, 'nonlinearli'),
             (25, 'superpixelwis'),
             (26, 'hallonquist'),
             (27, 'russakovski'),
             (28, 'de

In [51]:
palabras_tres_caracteres

OrderedDict([('www', 1),
             ('ieee', 5),
             ('ccce', 1),
             ('ieeexplor', 1),
             ('judddb', 25),
             ('iii', 1),
             ('aaai', 4),
             ('viii', 2),
             ('lll', 4),
             ('ieeecomputersocieti', 1),
             ('sss', 2),
             ('lsss', 22),
             ('xviii', 1),
             ('hhh', 1),
             ('aaa', 1),
             ('eccc', 4),
             ('adddit', 1),
             ('nnnnnnn', 4),
             ('xxxx', 2),
             ('sssr', 1),
             ('lnnn', 1),
             ('xxxxx', 1),
             ('jjj', 2),
             ('iiit', 1),
             ('xxx', 1),
             ('classsourc', 1),
             ('cofficlasssourc', 1),
             ('dalvikclasssourc', 1),
             ('eee', 1),
             ('thisssyntax', 1),
             ('ccc', 1),
             ('cccp', 1),
             ('ppp', 35),
             ('ssss', 1),
             ('ppppp', 2),
             ('pppp', 2),
      

# Cosas Varias

In [ ]:
#Buscar documentos con X palabra
palabras
raras = []
#bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb
for documento, words in bag_of_words.items():
    if 'integrationsreihenfolg' in words:
        raras.append(documento)

#buscar palabrasde mas de 15 caracteres       
for palabra in palabras:
    if len(palabra) > 15:
        print(palabra)

In [ ]:
#/opt/datasets/mcda-pi1-20191/papers-txt/1501.02741.txt
bag_of_words['/opt/datasets/mcda-pi1-20191/papers-txt/1501.02741.txt']['user']

input_file = open('/opt/datasets/mcda-pi1-20191/papers-txt/1501.02741.txt',"r",encoding='utf-8')
texto = input_file.read()
#texto = re.sub('[^A-Za-z0-9]+',' ',texto) # Caracteres especiales
tokens = texto.split()
stemmer = PorterStemmer()
stemmer2 = LancasterStemmer()
tokens = [wordnet_lemmatizer.lemmatize(stemmer.stem(w.lower()), pos="v") for w in tokens if (len(w)>1) and w.isalpha() and w not in sw] # Longitud mayor a 1
counter=collections.Counter(tokens)
dict(counter)['einhäuser']

In [10]:
#Traductor
def detect_language(text):

    languages_ratios = {}
    tokens = tokens = texto.split()
    tokens = [w.lower() for w in tokens]
    # Compute per language included in nltk number of unique stopwords appearing in analyzed text
    for language in stopwords.fileids():
        stopwords_set = set(stopwords.words(language))
        words_set = set(tokens)
        common_elements = words_set.intersection(stopwords_set)
        languages_ratios[language] = len(common_elements) # language "score"
    return max(languages_ratios, key=languages_ratios.get)


#----------------------------------------------------------------------
input_file = open(files_txt[0],"r",encoding='utf-8')
texto = input_file.read()
language = detect_language(texto)
language

'english'